# Neural Networks Task

In [71]:
import numpy as np
import tensorflow as tf

**code version 1**

In [72]:
N, D, H = 64, 1000, 100
x = tf.placeholder(tf.float32, shape = (N, D))                               # 64 x 1000 placeholder   (input)
y = tf.placeholder(tf.float32, shape = (N, D))                               # 64 x 1000 placeholder   (output)
w1 = tf.placeholder(tf.float32, shape = (D, H))                              # 1000 x 100 placeholder   (weights)
w2 = tf.placeholder(tf.float32, shape = (H, D))                              # 100 x 1000 placeholder   (weights)

h = tf.maximum(tf.matmul(x, w1), 0)                                          # Temporary variable contains the result of multiplyint x and w1
y_pred = tf.matmul(h, w2)                                                    # The predicted values of the output
diff = y_pred - y                                                            # Calculating the difference between the predicted and the real values of the output
loss = tf.reduce_mean(tf.reduce_sum(diff**2, axis = 1))                      # Calculating the loss function by using the mean square error method
grad_w1, grad_w2 = tf.gradients(loss, [w1, w2])                              # Calculate the loss of gradinet with respect to w1 and w2

with tf.Session() as sess:                                                   # Opening a session
    values = {x: np.random.randn(N, D),                                      # Assigning a random values for the placeholders
              w1: np.random.randn(D, H),
              w2: np.random.randn(H, D),
              y: np.random.randn(N, D)}
    out = sess.run([loss, grad_w1, grad_w2], feed_dict = values)            # Running the computational graph
    loss_val, grad_w1_val, grad_w2_val = out                                # Assigning the output to variables

print(loss_val)                                                             # Printing the results

50063464.0


**code version 2**

In [73]:
N, D, H = 64, 1000, 100
x = tf.placeholder(tf.float32, shape = (N, D))                               # 64 x 1000 placeholder   (input)
y = tf.placeholder(tf.float32, shape = (N, D))                               # 64 x 1000 placeholder   (output)
w1 = tf.placeholder(tf.float32, shape = (D, H))                              # 1000 x 100 placeholder   (weights)
w2 = tf.placeholder(tf.float32, shape = (H, D))                              # 100 x 1000 placeholder   (weights)

h = tf.maximum(tf.matmul(x, w1), 0)                                          # Temporary variable contains the result of multiplyint x and w1
y_pred = tf.matmul(h, w2)                                                    # The predicted values of the output
diff = y_pred - y                                                            # Calculating the difference between the predicted and the real values of the output
loss = tf.reduce_mean(tf.reduce_sum(diff**2, axis = 1))                      # Calculating the loss function by using the mean square error method
grad_w1, grad_w2 = tf.gradients(loss, [w1, w2])                              # Calculate the loss of gradinet with respect to w1 and w2

with tf.Session() as sess:                                                   # Opening a session
    values = {x: np.random.randn(N, D),                                      # Assigning a random values for the placeholders
              w1: np.random.randn(D, H),
              w2: np.random.randn(H, D),
              y: np.random.randn(N, D)}
    learning_rate = 1e-5                                                    # Assigne the learning rate
    for t in range(50):                                                     # traning loop with 50 iterations
        out = sess.run([loss, grad_w1, grad_w2], feed_dict = values)        # Running the computational graph
        loss_val, grad_w1_val, grad_w2_val = out                            # Assigning the output to variables
        values[w1] -= learning_rate*grad_w1_val                             # Use gradients to update weights
        values[w2] -= learning_rate*grad_w2_val                             # Use gradients to update weights
        print(loss_val)                                                     # Printing the results

50311012.0
17496160.0
9238482.0
5465027.0
3460670.5
2294339.5
1574057.5
1108183.8
795969.6
580745.4
428945.5
320130.84
240870.0
182514.86
139210.42
106787.39
82355.016
63812.168
49668.273
38813.83
30483.523
24050.203
19067.902
15185.591
12161.664
9797.135
7943.6104
6487.9326
5341.41
4437.5566
3723.481
3158.122
2710.4688
2355.376
2073.354
1849.0929
1670.6313
1528.4365
1415.0868
1324.685
1252.5925
1195.0593
1149.1765
1112.5808
1083.3813
1060.1294
1041.6132
1026.8856
1015.18634
1005.9175


**The code in the previous cell has a problem of copying weights between CPU and GPU each step, so we will do:**

**code version 3**

In [75]:
N, D, H = 64, 1000, 100
x = tf.placeholder(tf.float32, shape = (N, D))                               # 64 x 1000 placeholder   (input)
y = tf.placeholder(tf.float32, shape = (N, D))                               # 64 x 1000 placeholder   (output)
w1 = tf.Variable(tf.random_normal((D, H)))                                   # 1000 x 100 variable   (weights)
w2 = tf.Variable(tf.random_normal((H, D)))                                   # 100 x 1000 variable   (weights)

h = tf.maximum(tf.matmul(x, w1), 0)                                          #**code version 1** Temporary variable contains the result of multiplyint x and w1
y_pred = tf.matmul(h, w2)                                                    # The predicted values of the output
diff = y_pred - y                                                            # Calculating the difference between the predicted and the real values of the output
loss = tf.reduce_mean(tf.reduce_sum(diff**2, axis = 1))                      # Calculating the loss function by using the mean square error method
grad_w1, grad_w2 = tf.gradients(loss, [w1, w2])                              # Calculate the loss of gradinet with respect to w1 and w2

learning_rate = 1e-5                                                         # Assigne the learning rate
new_w1 = w1.assign(w1 - learning_rate*grad_w1)                               # Update w1
new_w2 = w2.assign(w2 - learning_rate*grad_w2)                               # Update w1

with tf.Session() as sess:                                                   # Opening a session
    sess.run(tf.global_variables_initializer())                              # Initialize the variables
    values = {x: np.random.randn(N, D),                                      # Assigning a random values for the placeholders
              y: np.random.randn(N, D)}
    for t in range(50):                                                      # traning loop with 50 iterations
        loss_val, = sess.run([loss], feed_dict = values)                     # Running the computational graph    
        print(loss_val)                                                      # Printing the results

53133656.0
53133656.0
53133656.0
53133656.0
53133656.0
53133656.0
53133656.0
53133656.0
53133656.0
53133656.0
53133656.0
53133656.0
53133656.0
53133656.0
53133656.0
53133656.0
53133656.0
53133656.0
53133656.0
53133656.0
53133656.0
53133656.0
53133656.0
53133656.0
53133656.0
53133656.0
53133656.0
53133656.0
53133656.0
53133656.0
53133656.0
53133656.0
53133656.0
53133656.0
53133656.0
53133656.0
53133656.0
53133656.0
53133656.0
53133656.0
53133656.0
53133656.0
53133656.0
53133656.0
53133656.0
53133656.0
53133656.0
53133656.0
53133656.0
53133656.0


**The code in the previous cell has a problem that the loss not going down each step, so we will do:**

**code version 4**

In [76]:
N, D, H = 64, 1000, 100
x = tf.placeholder(tf.float32, shape = (N, D))                               # 64 x 1000 placeholder   (input)
y = tf.placeholder(tf.float32, shape = (N, D))                               # 64 x 1000 placeholder   (output)
w1 = tf.Variable(tf.random_normal((D, H)))                                   # 1000 x 100 variable   (weights)
w2 = tf.Variable(tf.random_normal((H, D)))                                   # 100 x 1000 variable   (weights)

h = tf.maximum(tf.matmul(x, w1), 0)                                          # Temporary variable contains the result of multiplyint x and w1
y_pred = tf.matmul(h, w2)                                                    # The predicted values of the output
diff = y_pred - y                                                            # Calculating the difference between the predicted and the real values of the output
loss = tf.reduce_mean(tf.reduce_sum(diff**2, axis = 1))                      # Calculating the loss function by using the mean square error method
grad_w1, grad_w2 = tf.gradients(loss, [w1, w2])                              # Calculate the loss of gradinet with respect to w1 and w2

learning_rate = 1e-5                                                         # Assigne the learning rate
new_w1 = w1.assign(w1 - learning_rate*grad_w1)                               # Update w1
new_w2 = w2.assign(w2 - learning_rate*grad_w2)                               # Update w1
updates = tf.group(new_w1, new_w2)                                           # Putting new_w1 and new_w2 into one place

with tf.Session() as sess:                                                   # Opening a session
    sess.run(tf.global_variables_initializer())                              # Initialize the variables
    values = {x: np.random.randn(N, D),                                      # Assigning a random values for the placeholders
              y: np.random.randn(N, D)}
    losses = []                                                              # Empty place
    for t in range(50):                                                      # training loop with 50 iterations
        loss_val, _ = sess.run([loss, updates], feed_dict = values)          # run the computational graph
        print(loss_val)                                                      # print the loss values

49072380.0
17083582.0
8829200.0
5111463.0
3167654.8
2058837.5
1384816.5
956465.25
674228.3
483212.97
350803.8
257489.6
190767.16
142252.47
106909.664
80874.54
61570.1
47175.312
36362.14
28192.11
21986.258
17241.379
13606.946
10816.289
8662.755
6995.782
5702.322
4695.1196
3908.4612
3292.393
2808.6128
2428.1128
2128.1301
1891.1384
1703.5667
1554.8309
1436.6846
1342.7344
1267.9481
1208.3622
1160.8772
1123.0078
1092.8115
1068.6895
1049.4131
1034.0033
1021.69006
1011.82336
1003.9459
997.6482


**code version 5**

In [85]:
N, D, H = 64, 1000, 100
x = tf.placeholder(tf.float32, shape = (N, D))                               # 64 x 1000 placeholder   (input)
y = tf.placeholder(tf.float32, shape = (N, D))                               # 64 x 1000 placeholder   (output)
w1 = tf.Variable(tf.random_normal((D, H)))                                   # 1000 x 100 variable   (weights)
w2 = tf.Variable(tf.random_normal((H, D)))                                   # 100 x 1000 variable   (weights)

h = tf.maximum(tf.matmul(x, w1), 0)                                          # Temporary variable contains the result of multiplyint x and w1
y_pred = tf.matmul(h, w2)                                                    # The predicted values of the output
diff = y_pred - y                                                            # Calculating the difference between the predicted and the real values of the output
loss = tf.reduce_mean(tf.reduce_sum(diff*diff, axis = 1))                    # Calculating the loss function by using the mean square error method

optimizer = tf.train.GradientDescentOptimizer(1e-5)                          # Use an optimizer to compute gradients
updates = optimizer.minimize(loss)                                           # Update weights

with tf.Session() as sess:                                                   # Opening a session
    sess.run(tf.global_variables_initializer())                              # Initialize the variables
    values = {x: np.random.randn(N, D),                                      # Assigning a random values for the placeholders
              y: np.random.randn(N, D)}
    losses = []                                                              # Empty place
    for t in range(50):                                                      # training loop with 50 iterations
        loss_val, _ = sess.run([loss, updates], feed_dict = values)          # run the computational graph
        print(loss_val)                                                      # print the loss values

53387330.0
18333184.0
9645373.0
5710844.0
3627635.5
2419914.0
1672197.2
1186658.8
859759.0
632491.4
471683.06
355541.06
270281.56
207202.36
159935.81
124194.945
96922.68
76020.52
59893.07
47387.414
37636.203
30010.367
24024.918
19303.77
15575.615
12621.633
10275.998
8420.737
6946.715
5771.796
4832.9614
4081.2656
3478.3884
2994.166
2604.7249
2291.1152
2038.2766
1834.2617
1669.4802
1536.2458
1428.4622
1341.186
1270.3828
1213.1931
1166.895
1129.4792
1099.2377
1074.8423
1055.1044
1039.1637


**code version 6**

In [88]:
N, D, H = 64, 1000, 100
x = tf.placeholder(tf.float32, shape = (N, D))                               # 64 x 1000 placeholder   (input)
y = tf.placeholder(tf.float32, shape = (N, D))                               # 64 x 1000 placeholder   (output)
w1 = tf.Variable(tf.random_normal((D, H)))                                   # 1000 x 100 variable   (weights)
w2 = tf.Variable(tf.random_normal((H, D)))                                   # 100 x 1000 variable   (weights)

h = tf.maximum(tf.matmul(x, w1), 0)                                          # Temporary variable contains the result of multiplyint x and w1
y_pred = tf.matmul(h, w2)                                                    # The predicted values of the output
loss = tf.losses.mean_squared_error(y_pred, y)                               # use predefined common losses

optimizer = tf.train.GradientDescentOptimizer(1e-3)                          # Use an optimizer to compute gradients
updates = optimizer.minimize(loss)                                           # Update weights

with tf.Session() as sess:                                                   # Opening a session
    sess.run(tf.global_variables_initializer())                              # Initialize the variables
    values = {x: np.random.randn(N, D),                                      # Assigning a random values for the placeholders
              y: np.random.randn(N, D)}
    losses = []                                                              # Empty place
    for t in range(50):                                                      # training loop with 50 iterations
        loss_val, _ = sess.run([loss, updates], feed_dict = values)          # run the computational graph
        print(loss_val)                                                      # print the loss values

53310.562
48207.816
43829.438
40033.434
36713.574
33786.848
31189.867
28871.877
26792.12
24917.72
23221.557
21680.787
20276.47
18992.629
17815.58
16733.842
15737.347
14817.33
13966.456
13177.886
12445.686
11764.732
11130.816
10539.682
9987.816
9471.885
8989.078
8536.686
8112.398
7714.098
7339.6826
6987.518
6655.932
6343.5454
6048.941
5770.768
5507.9653
5259.644
5024.808
4802.4927
4592.002
4392.574
4203.485
4024.1294
3853.9
3692.2598
3538.6858
3392.6526
3253.7715
3121.5854


**code version 7**

In [89]:
N, D, H = 64, 1000, 100
x = tf.placeholder(tf.float32, shape = (N, D))                               # 64 x 1000 placeholder   (input)
y = tf.placeholder(tf.float32, shape = (N, D))                               # 64 x 1000 placeholder   (output)

init = tf.variance_scaling_initializer(2.0)                                  # use the intitializer
h = tf.layers.dense(inputs = x, units = H, activation = tf.nn.relu, kernel_initializer = init)     # Define the input layer
y_pred = tf.layers.dense(inputs = h, units = D, kernel_initializer = init)                        # Define the output layer

loss = tf.losses.mean_squared_error(y_pred, y)                               # use predefined common losses

optimizer = tf.train.GradientDescentOptimizer(1e0)                           # Use an optimizer to compute gradients
updates = optimizer.minimize(loss)                                           # Update weights

with tf.Session() as sess:                                                   # Opening a session
    sess.run(tf.global_variables_initializer())                              # Initialize the variables
    values = {x: np.random.randn(N, D),                                      # Assigning a random values for the placeholders
              y: np.random.randn(N, D)}
    for t in range(50):                                                      # training loop with 50 iterations
        loss_val, _ = sess.run([loss, updates], feed_dict = values)          # run the computational graph
        print(loss_val)                                                      # print the loss values

2.9078853
1.214218
1.0278909
0.9896201
0.98079515
0.97844297
0.9776362
0.97729117
0.97706896
0.97688353
0.9767067
0.9765339
0.9763623
0.9761911
0.97602004
0.9758491
0.9756782
0.9755073
0.97533643
0.97516555
0.9749946
0.9748236
0.9746528
0.9744818
0.9743109
0.97413987
0.9739689
0.973798
0.973627
0.97345597
0.9732848
0.97311383
0.97294277
0.9727716
0.97260046
0.9724292
0.9722581
0.9720869
0.97191554
0.97174424
0.971573
0.9714016
0.9712301
0.9710587
0.9708872
0.97071576
0.9705441
0.97037244
0.9702008
0.9700292


**code version 8**

In [90]:
N, D, H = 64, 1000, 100
x = tf.placeholder(tf.float32, shape = (N, D))                               # 64 x 1000 placeholder   (input)
y = tf.placeholder(tf.float32, shape = (N, D))                               # 64 x 1000 placeholder   (output)

model = tf.keras.Sequential()                                                # Build the neural network model
model.add(tf.keras.layers.Dense(H, input_shape = (D, ), activation = tf.nn.relu))     # Add a layer to the model
model.add(tf.keras.layers.Dense(D))                                                   # Add a layer to the model
y_pred = model(x)                                                            # Calculate thr predicted output (get output by calling the model), note no computation only building the computational graph
loss = tf.losses.mean_squared_error(y_pred, y)                               # use predefined common losses

optimizer = tf.train.GradientDescentOptimizer(1e0)                           # Use an optimizer to compute gradients
updates = optimizer.minimize(loss)                                           # Update weights

with tf.Session() as sess:                                                   # open session
    sess.run(tf.global_variables_initializer())                              # Initialize the variales
    values = {x: np.random.randn(N, D),                                      # Assigning a random values for the placeholders
              y: np.random.randn(N, D)}
    for t in range(50):                                                      # Training loop with 50 iterations
        loss_val, _ = sess.run([loss, updates], feed_dict = values)          # Calculate the losses
        print(loss_val)                                                      # Print the losses

1.1685723
1.1308577
1.1016709
1.0783638
1.0593646
1.0436316
1.0304482
1.0193079
1.0097802
1.0015754
0.99442506
0.9881281
0.9825398
0.97754323
0.9730388
0.9689579
0.9652262
0.96178466
0.958587
0.95558083
0.95274866
0.9500563
0.9474765
0.94498223
0.9425532
0.94018227
0.9378467
0.93554235
0.9332591
0.93098825
0.9287213
0.9264533
0.92417836
0.9218898
0.9195886
0.91726583
0.9149236
0.9125605
0.9101723
0.90775406
0.9053065
0.902829
0.90031886
0.8977759
0.89519787
0.89258456
0.8899349
0.88724595
0.88451576
0.8817465


**code version 9**

In [91]:
N, D, H = 64, 1000, 100
model = tf.keras.Sequential()                                                         # Build the neural network model
model.add(tf.keras.layers.Dense(H, input_shape = (D, ), activation = tf.nn.relu))     # Add a layer to the model
model.add(tf.keras.layers.Dense(D))                                                   # Add a layer to the model

model.compile(loss = tf.keras.losses.mean_squared_error, optimizer = tf.keras.optimizers.SGD(lr = 1e0))     # Define the loss and the optimizer

x = np.random.randn(N, D)                                                             # Assigning a random values for x
y = np.random.randn(N, D)                                                             # Assigning a random values for y

history = model.fit(x, y, epochs = 50, batch_size = N)                                # Training loop with 50 iterations, and batch size 64

Epoch 1/50
64/64 [==============================] - 1s 11ms/sample - loss: 1.1595
Epoch 2/50
64/64 [==============================] - 0s 244us/sample - loss: 1.1220
Epoch 3/50
64/64 [==============================] - 0s 0s/sample - loss: 1.0927
Epoch 4/50
64/64 [==============================] - 0s 244us/sample - loss: 1.0692
Epoch 5/50
64/64 [==============================] - 0s 244us/sample - loss: 1.0500
Epoch 6/50
64/64 [==============================] - 0s 78us/sample - loss: 1.0341
Epoch 7/50
64/64 [==============================] - 0s 156us/sample - loss: 1.0207
Epoch 8/50
64/64 [==============================] - 0s 94us/sample - loss: 1.0094
Epoch 9/50
64/64 [==============================] - 0s 188us/sample - loss: 0.9997
Epoch 10/50
64/64 [==============================] - 0s 172us/sample - loss: 0.9913
Epoch 11/50
64/64 [==============================] - 0s 78us/sample - loss: 0.9840
Epoch 12/50
64/64 [==============================] - 0s 156us/sample - loss: 0.9775
Epoch 13